# 资源分配问题建模

考虑三个工作岗位：测试员、Java开发人员和架构师。

考虑三个资源：Carlos、Joe和Monika。

## 数据 

每个资源执行各项工作的能力由以下匹配分数表说明：

![资源分配问题数据图](util/rap_data.png)


**假设**：每个工作只能分配一个资源，每个资源最多可以分配一项工作。

## 问题陈述

确定一种分配方案，确保每项工作都能得到完成，每个资源最多分配一项工作，以最大化分配的总匹配分数。

## 决策变量

决策变量 $x_{r,\; j} = 1$ 表示资源r被分配给工作j，否则为0，其中 r=1,2,3 且 𝑗=1,2,3。

## 约束条件

### 工作约束

对于每个工作 𝑗=1,2,3，必须从r=1,2,3中恰好分配一个资源。

约束（测试员=1）：$x_{1,\; 1} + x_{2,\; 1} + x_{3,\; 1} = 1$

约束（Java开发人员=2）：$x_{1,\; 2} + x_{2,\; 2} + x_{3,\; 2} = 1$

约束（架构师=3）：$x_{1,\; 3} + x_{2,\; 3} + x_{3,\; 3} = 1$

### 资源约束

对于每个资源 r=1,2,3，最多可以从j=1,2,3中分配一项工作。

约束（Carlos=1）：$x_{1,\; 1} + x_{1,\; 2} + x_{1,\; 3}  \leq 1$

约束（Joe=2）：$x_{2,\; 1} + x_{2,\; 2} + x_{2,\; 3}  \leq 1$

约束（Monika=3）：$x_{2,\; 1} + x_{2,\; 2} + x_{2,\; 3}  \leq 1$

## 目标函数

目标函数是在满足工作和资源约束的前提下，最大化分配的总匹配分数。

$$
Max \; (53x_{1,\; 1} + 80x_{2,\; 1} + 53x_{3,\; 1}) + (27x_{1,\; 2} + 47x_{2,\; 2} + 73x_{3,\; 2})
+ (13x_{1,\; 3} + 67x_{2,\; 3} + 47x_{3,\; 3})
$$




首先，按需安装gurobipy

In [ ]:
%pip install gurobipy

In [1]:
# 导入gurobi库
from gurobipy import *

## 数据
R列表包含了三个资源的名称：Carlos、Joe和Monika。

J列表包含了工作岗位的名称：测试员、Java开发人员和架构师

In [2]:
# 资源和工作集合
R = ['Carlos', 'Joe', 'Monika']
J = ['Tester', 'JavaDeveloper', 'Architect']

$r \in R$ 资源的索引和集合。

$j \in J$ 工作的索引和集合。

以下"multidict"函数描述了与每个资源和工作的可能组合相关的匹配分数

In [3]:
# 匹配分数数据
combinations, ms = multidict({
    ('Carlos', 'Tester'): 53,
    ('Carlos', 'JavaDeveloper'): 27,
    ('Carlos', 'Architect'): 13,
    ('Joe', 'Tester'): 80,
    ('Joe', 'JavaDeveloper'): 47,
    ('Joe', 'Architect'): 67,
    ('Monika', 'Tester'): 53,
    ('Monika', 'JavaDeveloper'): 73,
    ('Monika', 'Architect'): 47
})

以下函数生成一个空的模型对象"m"，并将字符串"RAP"模型名称作为其参数。

In [6]:
# 声明并初始化模型
m = Model('RAP')

Set parameter LicenseID to value 2601452


## 决策变量

决策变量 $x_{r,\; j} = 1$ 表示资源r被分配给工作j，否则为0，对于 $r \in R$ 和 $j \in J $。

"addVars()"方法定义了模型对象"m"的决策变量。

In [7]:
# 为RAP模型创建决策变量
x = m.addVars(combinations, name="assign")

## 工作约束

对于每个工作 𝑗=1,2,3，必须从r=1,2,3中恰好分配一个资源。

约束（测试员=1）：$x_{1,\; 1} + x_{2,\; 1} + x_{3,\; 1} = 1$

约束（Java开发人员=2）：$x_{1,\; 2} + x_{2,\; 2} + x_{3,\; 2} = 1$

约束（架构师=3）：$x_{1,\; 3} + x_{2,\; 3} + x_{3,\; 3} = 1$

"addConstrs()"方法定义了模型对象"m"的约束条件。


In [8]:
# 创建工作约束
job = m.addConstrs((x.sum('*',j) == 1 for j in J), 'job')

## 工作约束
$$
\sum_{r \: \in \: R} x_{r,\; j} = 1 \; \; \; \forall \; j \in J
$$

## 资源约束

对于每个资源 r=1,2,3，最多可以从j=1,2,3中分配一项工作。

约束（Carlos=1）：$x_{1,\; 1} + x_{1,\; 2} + x_{1,\; 3}  \leq 1$

约束（Joe=2）：$x_{2,\; 1} + x_{2,\; 2} + x_{2,\; 3}  \leq 1$

约束（Monika=3）：$x_{3,\; 1} + x_{3,\; 2} + x_{3,\; 3}  \leq 1$

"addConstrs()"方法定义了模型对象"m"的约束条件。

In [9]:
# 创建资源约束
resource = m.addConstrs((x.sum(r,'*') <= 1 for r in R), 'resource')

## 资源约束
$$
\sum_{j \: \in \: J} x_{r,\; j} \leq 1 \; \; \; \forall \; r \in R
$$

## 目标函数

目标函数是最大化分配的总匹配分数。

$$
Max \; (53x_{1,\; 1} + 80x_{2,\; 1} + 53x_{3,\; 1}) + (27x_{1,\; 2} + 47x_{2,\; 2} + 73x_{3,\; 2})
+ (13x_{1,\; 3} + 67x_{2,\; 3} + 47x_{3,\; 3})
$$

"setObjective()"方法定义了模型对象"m"的目标函数。

In [12]:
# 目标是最大化分配的总匹配分数
m.setObjective(x.prod(ms), GRB.MAXIMIZE)

## 目标函数
注意
$$
(53x_{1,\; 1} + 80x_{2,\; 1} + 53x_{3,\; 1}) = \sum_{r \; \in \; R} ms_{r,1}x_{r,1} \\
(27x_{1,\; 2} + 47x_{2,\; 2} + 73x_{3,\; 2}) = \sum_{r \; \in \; R} ms_{r,2}x_{r,2} \\
(13x_{1,\; 3} + 67x_{2,\; 3} + 47x_{3,\; 3})  = \sum_{r \; \in \; R} ms_{r,3}x_{r,3}
$$

因此，目标函数可以表示为

$$
Max \; \sum_{j \; \in \; J} \sum_{r \; \in \; R} ms_{r,j}x_{r,j}
$$

In [11]:
# 保存模型以便检查
m.write('RAP.lp')

In [13]:
# 运行优化引擎
m.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) Ultra 5 125H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 18 logical processors, using up to 18 threads

Optimize a model with 6 rows, 9 columns and 18 nonzeros
Model fingerprint: 0xb343b6eb
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.01s
Presolved: 6 rows, 9 columns, 18 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.6000000e+32   1.800000e+31   4.600000e+02      0s
       5    1.9300000e+02   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.02 seconds (0.00 work units)
Optimal objective  1.930000000e+02


In [14]:
def print_solution(model):
    for var in model.getVars():
        if abs(var.x) > 1e-6:
            print("{0}: {1}".format(var.varName, var.x))
    print('总匹配分数: {0}'.format(model.objVal))
    return None

# 显示决策变量的最优值
print_solution(m)   


assign[Carlos,Tester]: 1.0
assign[Joe,Architect]: 1.0
assign[Monika,JavaDeveloper]: 1.0
总匹配分数: 193.0


## 新场景
考虑一个名为Ada的新资源，她对所有工作岗位的匹配分数都是100%。

## 约束条件

### 工作约束

对于每个工作 𝑗=1,2,3，必须从r=1,2,3,4中恰好分配一个资源。

约束（测试员=1）：$x_{1,\; 1} + x_{2,\; 1} + x_{3,\; 1} + x_{4,\; 1} = 1$

约束（Java开发人员=2）：$x_{1,\; 2} + x_{2,\; 2} + x_{3,\; 2} + x_{4,\; 2} = 1$

约束（架构师=3）：$x_{1,\; 3} + x_{2,\; 3} + x_{3,\; 3} + x_{4,\; 3}= 1$

### 资源约束

对于资源r=4，最多可以从j=1,2,3中分配一项工作。

约束（Ada=1）：$x_{4,\; 1} + x_{4,\; 2} + x_{4,\; 3}  \leq 1$


## 目标函数

目标函数是在满足工作和资源约束的前提下，最大化分配的总匹配分数。

$$
Max \; (53x_{1,\; 1} + 80x_{2,\; 1} + 53x_{3,\; 1} + 100x_{4, \; 1}) 
+ (27x_{1,\; 2} + 47x_{2,\; 2} + 73x_{3,\; 2} + 100x_{4, \; 2})
+ (13x_{1,\; 3} + 67x_{2,\; 3} + 47x_{3,\; 3} + 100x_{4, \; 3})
$$


In [15]:
new_scores = {('Ada','Tester'):100, ('Ada','JavaDeveloper'):100, ('Ada','Architect'):100}

for key, val in new_scores.items():
    r, j = key
    x[key] = m.addVar(obj=val,
                           name='assign[{0},{1}]'.format(r,j),
                           column=Column([1], [m.getConstrByName('job[{0}]'.format(j))]))
m.addConstr(x.sum('Ada','*') <= 1, name="resources[Ada]")

<gurobi.Constr *Awaiting Model Update*>

In [16]:
# 重新优化
m.optimize()
# 显示决策变量的最优值
print_solution(m)

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) Ultra 5 125H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 18 logical processors, using up to 18 threads

Optimize a model with 7 rows, 12 columns and 24 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
LP warm-start: use basis

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.4100000e+32   9.000000e+30   1.410000e+02      0s
       2    2.5300000e+02   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.02 seconds (0.00 work units)
Optimal objective  2.530000000e+02
assign[Joe,Tester]: 1.0
assign[Monika,JavaDeveloper]: 1.0
assign[Ada,Architect]: 1.0
总匹配分数: 253.0
